In [1]:
from __future__ import print_function
import paddle
import paddle.fluid as fluid
import numpy
import sys

In [2]:
# Each batch will yield 32 images
BATCH_SIZE = 128
EPOCH_NUM = 30
lr = 0.0001

use_cuda = True
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()


In [3]:
def vgg_bn_drop(input):
    def conv_block(ipt, num_filter, groups, dropouts):
        return fluid.nets.img_conv_group(
            input=ipt,
            pool_size=2,
            pool_stride=2,
            conv_num_filter=[num_filter] * groups,
            conv_filter_size=3,
            conv_act='relu',
            conv_with_batchnorm=True,
            conv_batchnorm_drop_rate=dropouts,
            pool_type='max')

    conv1 = conv_block(input, 64, 2, [0.3, 0])
    conv2 = conv_block(conv1, 128, 2, [0.4, 0])
    conv3 = conv_block(conv2, 256, 3, [0.4, 0.4, 0])
    conv4 = conv_block(conv3, 512, 3, [0.4, 0.4, 0])
    conv5 = conv_block(conv4, 512, 3, [0.4, 0.4, 0])

    drop = fluid.layers.dropout(x=conv5, dropout_prob=0.5)
    fc1 = fluid.layers.fc(input=drop, size=512, act=None)
    bn = fluid.layers.batch_norm(input=fc1, act='relu')
    drop2 = fluid.layers.dropout(x=bn, dropout_prob=0.5)
    fc2 = fluid.layers.fc(input=drop2, size=512, act=None)
    predict = fluid.layers.fc(input=fc2, size=10, act='softmax')
    return predict

In [4]:
def inference_program():
    # The image is 32 * 32 with RGB representation.
    data_shape = [3, 32, 32]
    images = fluid.layers.data(name='pixel', shape=data_shape, dtype='float32')

    predict = vgg_bn_drop(images)
    # predict = vgg_bn_drop(images) # un-comment to use vgg net
    return predict

In [5]:
def train_program():
    predict = inference_program()

    label = fluid.layers.data(name='label', shape=[1], dtype='int64')
    cost = fluid.layers.cross_entropy(input=predict, label=label)
    avg_cost = fluid.layers.mean(cost)
    accuracy = fluid.layers.accuracy(input=predict, label=label)
    return [avg_cost, accuracy]

In [6]:
def optimizer_program():
    return fluid.optimizer.Adam(learning_rate=lr)

In [7]:

# Reader for training
# image pixels in [0, 1] and label in [0, 99].
train_reader = paddle.batch(
    paddle.reader.shuffle(paddle.dataset.cifar.train10(), buf_size=50000),
    batch_size=BATCH_SIZE)

# Reader for testing. A separated data set for testing.
test_reader = paddle.batch(
    paddle.dataset.cifar.test10(), batch_size=BATCH_SIZE)

file 2ec5301f8df486a1d3decedc8f708bd6  md5 c58f30108f718f92721af3b95e74349a
Cache file /root/.cache/paddle/dataset/cifar/cifar-10-python.tar.gz not found, downloading https://dataset.bj.bcebos.com/cifar/cifar-10-python.tar.gz 
Begin to download
....................
Download finished


In [8]:

feed_order = ['pixel', 'label']

main_program = fluid.default_main_program()
star_program = fluid.default_startup_program()

avg_cost, acc = train_program()

# Test program
test_program = main_program.clone(for_test=True)

optimizer = optimizer_program()
optimizer.minimize(avg_cost)

exe = fluid.Executor(place)



# For training test cost
def train_test(program, reader):
    count = 0
    feed_var_list = [
        program.global_block().var(var_name) for var_name in feed_order
    ]
    feeder_test = fluid.DataFeeder(
        feed_list=feed_var_list, place=place)
    test_exe = fluid.Executor(place)
    accumulated = len([avg_cost, acc]) * [0]
    for tid, test_data in enumerate(reader()):
        avg_cost_np = test_exe.run(program=program,
                                   feed=feeder_test.feed(test_data),
                                   fetch_list=[avg_cost, acc])
        accumulated = [x[0] + x[1][0] for x in zip(accumulated, avg_cost_np)]
        count += 1
    return [x / count for x in accumulated]

In [9]:
params_dirname = "model/image_classification_vgg.inference.model"

from paddle.utils.plot import Ploter
import time

train_prompt = "Train cost"
test_prompt = "Test cost"
plot_cost = Ploter(test_prompt,train_prompt)
RESULT_FILE = 'result/results_vgg16.txt'

EPOCH_NUM = 30
print_every = 200
# main train loop.
def train_loop():
    feed_var_list_loop = [
        main_program.global_block().var(var_name) for var_name in feed_order
    ]
    feeder = fluid.DataFeeder(
        feed_list=feed_var_list_loop, place=place)
    exe.run(star_program)

    
    
    start_time = time.time()
    for pass_id in range(EPOCH_NUM):
        average_loss = 0
        step = 1
        for step_id, data_train in enumerate(train_reader()):
            avg_loss_value = exe.run(main_program,
                                     feed=feeder.feed(data_train),
                                     fetch_list=[avg_cost, acc])
            average_loss += avg_loss_value[0]
            if step % print_every == 0:
#                 plot_cost.append(train_prompt, step, avg_loss_value[0])
#                 plot_cost.plot()
                average_loss = average_loss /print_every
                print('[epoch: %3d, step: %5d] loss: %.3f' % (pass_id + 1, step, average_loss))
                average_loss = 0
            step += 1
            
    total_time = time.time() - start_time
    with open(RESULT_FILE, 'a') as f:
        f.write('\n\n\ntraining results: \n')
        f.write('\n total training time: \t {}'.format(total_time))
        f.write('\n final average training lostt: \t %.3f' % (average_loss))

    start_time = time.time()
    avg_cost_test, accuracy_test = train_test(test_program, reader=test_reader)
    total_time = time.time() - start_time
#     plot_cost.append(test_prompt, step, avg_cost_test)
    s = '\ntesting results: \n accuracy on 1000 test images: %.3f, total time: %d s\n' % (accuracy_test, total_time)
    print(s)
    with open(RESULT_FILE, 'a') as f:
        f.write(s)

    # save parameters
#     if params_dirname is not None:
#         fluid.io.save_inference_model(params_dirname, ["pixel"],
#                                         [predict], exe)

2020-07-20 03:12:37,987-INFO: font search path ['/usr/local/lib/python2.7/dist-packages/matplotlib/mpl-data/fonts/ttf', '/usr/local/lib/python2.7/dist-packages/matplotlib/mpl-data/fonts/afm', '/usr/local/lib/python2.7/dist-packages/matplotlib/mpl-data/fonts/pdfcorefonts']
2020-07-20 03:12:38,293-INFO: generated new fontManager


In [10]:
train_loop()

[epoch:   1, step:   200] loss: 2.305
[epoch:   2, step:   200] loss: 1.914
[epoch:   3, step:   200] loss: 1.723
[epoch:   4, step:   200] loss: 1.561
[epoch:   5, step:   200] loss: 1.443
[epoch:   6, step:   200] loss: 1.330
[epoch:   7, step:   200] loss: 1.237
[epoch:   8, step:   200] loss: 1.147
[epoch:   9, step:   200] loss: 1.095
[epoch:  10, step:   200] loss: 1.020
[epoch:  11, step:   200] loss: 0.975
[epoch:  12, step:   200] loss: 0.924
[epoch:  13, step:   200] loss: 0.881
[epoch:  14, step:   200] loss: 0.837
[epoch:  15, step:   200] loss: 0.804
[epoch:  16, step:   200] loss: 0.765
[epoch:  17, step:   200] loss: 0.747
[epoch:  18, step:   200] loss: 0.704
[epoch:  19, step:   200] loss: 0.665
[epoch:  20, step:   200] loss: 0.651
[epoch:  21, step:   200] loss: 0.624
[epoch:  22, step:   200] loss: 0.604
[epoch:  23, step:   200] loss: 0.580
[epoch:  24, step:   200] loss: 0.564
[epoch:  25, step:   200] loss: 0.560
[epoch:  26, step:   200] loss: 0.534
[epoch:  27,

In [11]:
train_loop()

[epoch:   1, step:   200] loss: 2.295
[epoch:   2, step:   200] loss: 1.914
[epoch:   3, step:   200] loss: 1.734
[epoch:   4, step:   200] loss: 1.561
[epoch:   5, step:   200] loss: 1.430
[epoch:   6, step:   200] loss: 1.335
[epoch:   7, step:   200] loss: 1.240
[epoch:   8, step:   200] loss: 1.170
[epoch:   9, step:   200] loss: 1.100
[epoch:  10, step:   200] loss: 1.036
[epoch:  11, step:   200] loss: 0.983
[epoch:  12, step:   200] loss: 0.936
[epoch:  13, step:   200] loss: 0.897
[epoch:  14, step:   200] loss: 0.844
[epoch:  15, step:   200] loss: 0.814
[epoch:  16, step:   200] loss: 0.771
[epoch:  17, step:   200] loss: 0.751
[epoch:  18, step:   200] loss: 0.718
[epoch:  19, step:   200] loss: 0.680
[epoch:  20, step:   200] loss: 0.667
[epoch:  21, step:   200] loss: 0.640
[epoch:  22, step:   200] loss: 0.621
[epoch:  23, step:   200] loss: 0.602
[epoch:  24, step:   200] loss: 0.573
[epoch:  25, step:   200] loss: 0.556
[epoch:  26, step:   200] loss: 0.539
[epoch:  27,

In [12]:
train_loop()

[epoch:   1, step:   200] loss: 2.250
[epoch:   2, step:   200] loss: 1.894
[epoch:   3, step:   200] loss: 1.720
[epoch:   4, step:   200] loss: 1.577
[epoch:   5, step:   200] loss: 1.451
[epoch:   6, step:   200] loss: 1.334
[epoch:   7, step:   200] loss: 1.242
[epoch:   8, step:   200] loss: 1.173
[epoch:   9, step:   200] loss: 1.087
[epoch:  10, step:   200] loss: 1.022
[epoch:  11, step:   200] loss: 0.969
[epoch:  12, step:   200] loss: 0.910
[epoch:  13, step:   200] loss: 0.875
[epoch:  14, step:   200] loss: 0.834
[epoch:  15, step:   200] loss: 0.806
[epoch:  16, step:   200] loss: 0.762
[epoch:  17, step:   200] loss: 0.737
[epoch:  18, step:   200] loss: 0.712
[epoch:  19, step:   200] loss: 0.678
[epoch:  20, step:   200] loss: 0.651
[epoch:  21, step:   200] loss: 0.621
[epoch:  22, step:   200] loss: 0.606
[epoch:  23, step:   200] loss: 0.576
[epoch:  24, step:   200] loss: 0.569
[epoch:  25, step:   200] loss: 0.545
[epoch:  26, step:   200] loss: 0.533
[epoch:  27,